In [ ]:
import requests
import json

f = open("redmine_info.txt", "w")
url = "http://localhost:3000/issues.json?project_id=testproject-redmine"
headers = {
    "X-Redmine-API-Key": "00b97c8f09f4e8498c720f0043b79930ef9c2823"
}

# response = requests.get(url, headers=headers)

# data = json.loads(response.text)
# print(json.dumps(data, indent=2, ensure_ascii=False))
# f.write(json.dumps(data, indent=2, ensure_ascii=False))
# f.close()

In [ ]:
# 1. 拉取 Redmine 工单
def redmine_get_issues(api_key, project_id, url_base="http://localhost:3000"):
    url = f"{url_base}/issues.json?project_id={project_id}"
    headers = {"X-Redmine-API-Key": api_key}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()["issues"]

# 2. 格式化工单列表为字符串
def format_redmine_issues_to_str(issues):
    lines = []
    for i in issues:
        subject = i.get("subject", "无标题")
        status = i.get("status", {}).get("name", "未知状态")
        priority = i.get("priority", {}).get("name", "无优先级")
        assignee = (i.get("assigned_to") or {}).get("name", "未分配")
        author = (i.get("author") or {}).get("name", "未知作者")
        start_date = i.get("start_date", "无开始日期")
        due_date = i.get("due_date", "无截止日期")
        created_on = i.get("created_on", "未知创建时间")
        updated_on = i.get("updated_on", "未知更新时间")
        desc = i.get("description", "无描述")
        lines.append(
            f"{subject} | 状态: {status} | 优先级: {priority} | 负责人: {assignee} | 作者: {author} | 开始: {start_date} | 截止: {due_date} | 创建: {created_on} | 更新: {updated_on} | 描述: {desc[:20]}..."
        )
    return "\n".join(lines)

In [ ]:
# 3. 获取 Redmine 工单并格式化
REDMINE_API_KEY = "00b97c8f09f4e8498c720f0043b79930ef9c2823"
REDMINE_PROJECT_ID = "testproject-redmine"

issues = redmine_get_issues(REDMINE_API_KEY, REDMINE_PROJECT_ID)
issues_str = format_redmine_issues_to_str(issues)
print("Formatted Redmine issues:")
print(issues_str)

In [ ]:
# 5. 调用 Gemini 做 Redmine 工单分析
from google import genai
from google.genai import types as gt
import os

# 假设你已设置好 GEMINI_API_KEY 环境变量
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI_API_KEY)

prompt = (
    "你是资深项目管理顾问，请根据以下 Redmine 工单列表，分析风险并提出对策：\n\n"
    + issues_str
)

print("\n=== Prompt to Gemini ===")
print(prompt)

resp = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=gt.GenerateContentConfig(temperature=0.3)
    )

# 6. 获取 Gemini 分析结果
gemini_response = resp.text
print("=== AI 分析结果 ===")
print(gemini_response)

In [ ]:
# 7. （可选）将 Gemini 分析结果写入本地文件
with open("redmine_gemini_analysis.txt", "w") as f:
    f.write(gemini_response)

In [ ]:
issue_id = 1  # 替换为你的工单ID
api_key = "00b97c8f09f4e8498c720f0043b79930ef9c2823"
redmine_url = "http://localhost:3000"
comment = "这里是AI分析建议内容" + gemini_response

url = f"{redmine_url}/issues/{issue_id}.json"
headers = {
    "X-Redmine-API-Key": api_key,
    "Content-Type": "application/json; charset=utf-8"
}
data = {
    "issue": {
        "notes": comment
    }
}

response = requests.put(
    url,
    data=json.dumps(data, ensure_ascii=False).encode("utf-8"),
    headers=headers
)
print(response.status_code)
if response.headers.get("Content-Type", "").startswith("application/json"):
    print(response.json())
else:
    print("不是JSON，原始内容如上")